# Email Spam detection EDA
Prepared By Deepa Francis<br>
For BrainStation<br>
On July 20, 2023

In [1]:
# import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from scipy.stats import norm

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

# display all columns in dataframe
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# loading the data
raw_data = pd.read_csv('raw_data.csv')

In [3]:
# Summary of raw data
print(f"Number of Rows: {raw_data.shape[0]}")
print(f"Number of Columns: {raw_data.shape[1]}")
print(f"Number of missing values: {raw_data.isna().sum().sum()}")
print(f"Number of duplicated rows: {raw_data.duplicated().sum()}")

Number of Rows: 75419
Number of Columns: 5
Number of missing values: 2856
Number of duplicated rows: 1512


In [4]:
# Quickly check the data types
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75419 entries, 0 to 75418
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       75419 non-null  int64 
 1   subject     74626 non-null  object
 2   email_to    74843 non-null  object
 3   email_from  75419 non-null  object
 4   message     73932 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.9+ MB
